In [2]:
# Import necessary libraries

!pip install ultralytics -q
# !pip install gradio -q
# !pip install pandas -q
!pip install deepface -q

^C
ERROR: Operation cancelled by user


: 

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
# import gradio as gr
import shutil
import pandas
import cv2
import os

In [ ]:
def faceRecognition(input_image):
    # Path to the directory containing cropped objects
    cropped_objects_dir = "./faces/"
    
    # Path to the directory to save unknown faces
    unknown_faces_dir = "./unknown/"
    
    # Path to the directory to save known faces
    known_faces_dir = "./known/"
    
    # Initialize a list to store the extracted names
    extracted_names = []
    
    # Check if the 'unknown' folder exists, otherwise create it
    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        # If the 'unknown' folder exists, clear all files and subfolders
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    # Check if the 'known' folder exists, otherwise create it
    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        # If the 'known' folder exists, clear all files and subfolders
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    # Iterate through the image files in the directory
    for filename in os.listdir(cropped_objects_dir):
        if filename.endswith(".jpg"):
            img_path = os.path.join(cropped_objects_dir, filename)
            model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

            # Check if a face was recognized in the image
            if model and len(model[0]['identity']) > 0:
                # Extract the name and append it to the list
                name = model[0]['identity'][0].split('/')[1]
                
                # Save the known face into the 'known' folder
                known_faces_path = os.path.join(known_faces_dir, f"{len(extracted_names) + 1}_{name}.jpg")
                shutil.copy(img_path, known_faces_path)
                
            else:
                # If no face is recognized, set name to 'unknown'
                name = 'unknown'
                
                # Save the unknown face into the 'unknown' folder
                unknown_faces_path = os.path.join(unknown_faces_dir, f"{len(extracted_names) + 1}.jpg")
                shutil.copy(img_path, unknown_faces_path)
                
            extracted_names.append(name)
            
    return extracted_names

In [ ]:
def faceExtraction(input_image, model, results):
    # Load the image
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

            detected_objects.append((object_name, (x1, y1, x2, y2)))

    # Create or clear the 'faces' directory
    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    # Crop and save each detected object
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        object_image.save(f"faces/face{i}.jpg")
        
    return 0

In [ ]:
def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]

    return faceExtraction(input_image, model, results)

In [ ]:
image = input("enter image path : ")
faceDetection(image)
names = faceRecognition(image)
print(names)


image 1/1 /Users/anirudh/Desktop/DS_environment/F_R/DSCN0797.JPG: 480x640 7 faces, 99.9ms
Speed: 9.1ms preprocess, 99.9ms inference, 10.2ms postprocess per image at shape (1, 3, 480, 640)
24-07-01 12:09:53 - Found 26 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 26/26 [02:00<00:00,  4.63s/it]


24-07-01 12:11:54 - There are now 275 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
24-07-01 12:11:54 - Searching ./faces/face1.jpg in 275 length datastore
24-07-01 12:11:54 - find function duration 120.68945622444153 seconds
24-07-01 12:11:54 - Searching ./faces/face0.jpg in 275 length datastore
24-07-01 12:11:54 - find function duration 0.2228379249572754 seconds
24-07-01 12:11:54 - Searching ./faces/face2.jpg in 275 length datastore
24-07-01 12:11:54 - find function duration 0.18482685089111328 seconds
24-07-01 12:11:54 - Searching ./faces/face3.jpg in 275 length datastore
24-07-01 12:11:55 - find function duration 0.17038536071777344 seconds
24-07-01 12:11:55 - Searching ./faces/face6.jpg in 275 length datastore
24-07-01 12:11:55 - find function duration 0.20537900924682617 seconds
24-07-01 12:11:55 - Searching ./faces/face4.jpg in 275 length datastore
24-07-01 12:11:55 - find function duration 0.21099328994750977 seconds
24-07-01 12

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
import shutil
import cv2
import os

def faceRecognition(input_image):
    cropped_objects_dir = "./faces/"
    unknown_faces_dir = "./unknown/"
    known_faces_dir = "./known/"
    extracted_names = []
    
    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    for filename in os.listdir(cropped_objects_dir):
        if filename.endswith(".jpg"):
            img_path = os.path.join(cropped_objects_dir, filename)
            model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

            if model and len(model[0]['identity']) > 0:
                name = model[0]['identity'][0].split('/')[1]
                known_faces_path = os.path.join(known_faces_dir, f"{len(extracted_names) + 1}_{name}.jpg")
                shutil.copy(img_path, known_faces_path)
            else:
                name = 'unknown'
                unknown_faces_path = os.path.join(unknown_faces_dir, f"{len(extracted_names) + 1}.jpg")
                shutil.copy(img_path, unknown_faces_path)
                
            extracted_names.append(name)
            
    return extracted_names

def faceExtraction(input_image, model, results):
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            detected_objects.append((object_name, (x1, y1, x2, y2)))

    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        object_image.save(f"faces/face{i}.jpg")
        
    return detected_objects

def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]
    return faceExtraction(input_image, model, results)

def capture_and_recognize():
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image from webcam")
            break

        frame_path = "current_frame.jpg"
        cv2.imwrite(frame_path, frame)

        detected_faces = faceDetection(frame_path)
        
        if detected_faces:
            names = faceRecognition(frame_path)
            for name in names:
                print(name)

            for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_faces):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, names[i], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        else:
            print("No faces detected")

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cam.release()
    cv2.destroyAllWindows()

# Start real-time face recognition
capture_and_recognize()


In [2]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
import shutil
import cv2
import os

def faceRecognition(face_images):
    unknown_faces_dir = "./unknown/"
    known_faces_dir = "./known/"
    extracted_names = []

    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    for i, face_image in enumerate(face_images):
        img_path = face_image
        model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

        if model and len(model[0]['identity']) > 0:
            name = model[0]['identity'][0].split('/')[1]
            known_faces_path = os.path.join(known_faces_dir, f"{i + 1}_{name}.jpg")
            shutil.copy(img_path, known_faces_path)
        else:
            name = 'unknown'
            unknown_faces_path = os.path.join(unknown_faces_dir, f"{i + 1}.jpg")
            shutil.copy(img_path, unknown_faces_path)
            
        extracted_names.append(name)
    
            
    return extracted_names

def faceExtraction(input_image, model, results):
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            detected_objects.append((object_name, (x1, y1, x2, y2)))

    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    face_images = []
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        face_path = f"faces/face{i}.jpg"
        object_image.save(face_path)
        face_images.append(face_path)
        
    return detected_objects, face_images

def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]
    return faceExtraction(input_image, model, results)

def capture_and_recognize():
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image from webcam")
            break

        frame_path = "current_frame.jpg"
        cv2.imwrite(frame_path, frame)

        detected_faces, face_images = faceDetection(frame_path)
        
        if detected_faces:
            names = faceRecognition(face_images)
            for name in names:
                print(name)

            for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_faces):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, names[i], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        else:
            print("No faces detected")

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cam.release()
    cv2.destroyAllWindows()

# Start real-time face recognition
capture_and_recognize()



image 1/1 /Users/anirudh/Desktop/DS_environment/F_R/current_frame.jpg: 480x640 1 face, 129.3ms
Speed: 2.3ms preprocess, 129.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


KeyboardInterrupt: 

In [1]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
import shutil
import cv2
import os

def faceRecognition(face_images, threshold=0.4):
    unknown_faces_dir = "./unknown/"
    known_faces_dir = "./known/"
    extracted_names = []
    confidences = []

    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    for i, face_image in enumerate(face_images):
        img_path = face_image
        model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

        if model and len(model[0]['identity']) > 0:
            distance = model[0]['distance'][0]
            if distance < threshold:
                name = model[0]['identity'][0].split('/')[1]
                known_faces_path = os.path.join(known_faces_dir, f"{i + 1}_{name}.jpg")
                shutil.copy(img_path, known_faces_path)
                extracted_names.append(name)
                confidences.append(1 - distance)  # Convert distance to confidence (higher is better)
            else:
                name = 'unknown'
                extracted_names.append(name)
                confidences.append(None)
        else:
            name = 'unknown'
            confidence = None
            unknown_faces_path = os.path.join(unknown_faces_dir, f"{i + 1}.jpg")
            shutil.copy(img_path, unknown_faces_path)
            extracted_names.append(name)
            confidences.append(confidence)
            
    return extracted_names, confidences

def faceExtraction(input_image, model, results):
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            detected_objects.append((object_name, (x1, y1, x2, y2)))

    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    face_images = []
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        face_path = f"faces/face{i}.jpg"
        object_image.save(face_path)
        face_images.append(face_path)
        
    return detected_objects, face_images

def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]
    return faceExtraction(input_image, model, results)

def capture_and_recognize():
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image from webcam")
            break

        frame_path = "current_frame.jpg"
        cv2.imwrite(frame_path, frame)

        detected_faces, face_images = faceDetection(frame_path)
        
        if detected_faces:
            names, confidences = faceRecognition(face_images)
            for name, confidence in zip(names, confidences):
                print(f"{name}: {confidence}")

            for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_faces):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                if confidences[i] is not None:
                    label = f"{names[i]} ({confidences[i]:.2f})"
                else:
                    label = names[i]
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        else:
            print("No faces detected")

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(30) & 0xFF == 27:
            break

    cam.release()
    cv2.destroyAllWindows()

# Start real-time face recognition
capture_and_recognize()



image 1/1 /Users/anirudh/Desktop/DS_environment/F_R/current_frame.jpg: 480x640 1 face, 102.7ms
Speed: 3.7ms preprocess, 102.7ms inference, 8.2ms postprocess per image at shape (1, 3, 480, 640)


KeyboardInterrupt: 

In [1]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
import shutil
import cv2
import os
import csv
import datetime
import time

def faceRecognition(face_images, threshold=0.4):
    unknown_faces_dir = "./unknown/"
    known_faces_dir = "./known/"
    extracted_names = []
    confidences = []

    if not os.path.exists(unknown_faces_dir):
        os.makedirs(unknown_faces_dir)
    else:
        for file_or_folder in os.listdir(unknown_faces_dir):
            file_or_folder_path = os.path.join(unknown_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    if not os.path.exists(known_faces_dir):
        os.makedirs(known_faces_dir)
    else:
        for file_or_folder in os.listdir(known_faces_dir):
            file_or_folder_path = os.path.join(known_faces_dir, file_or_folder)
            if os.path.isfile(file_or_folder_path):
                os.remove(file_or_folder_path)
            elif os.path.isdir(file_or_folder_path):
                shutil.rmtree(file_or_folder_path)

    for i, face_image in enumerate(face_images):
        img_path = face_image
        model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

        if model and len(model[0]['identity']) > 0:
            distance = model[0]['distance'][0]
            if distance < threshold:
                name = model[0]['identity'][0].split('/')[1]
                known_faces_path = os.path.join(known_faces_dir, f"{i + 1}_{name}.jpg")
                shutil.copy(img_path, known_faces_path)
                extracted_names.append(name)
                confidences.append(1 - distance)  # Convert distance to confidence (higher is better)
            else:
                name = 'unknown'
                extracted_names.append(name)
                confidences.append(None)
        else:
            name = 'unknown'
            confidence = None
            unknown_faces_path = os.path.join(unknown_faces_dir, f"{i + 1}.jpg")
            shutil.copy(img_path, unknown_faces_path)
            extracted_names.append(name)
            confidences.append(confidence)
            
    return extracted_names, confidences

def faceExtraction(input_image, model, results):
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            detected_objects.append((object_name, (x1, y1, x2, y2)))

    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    face_images = []
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        face_path = f"faces/face{i}.jpg"
        object_image.save(face_path)
        face_images.append(face_path)
        
    return detected_objects, face_images

def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]
    return faceExtraction(input_image, model, results)

def save_to_csv(name, manager, group):
    date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    csv_filename = f"{date_str}.csv"
    fieldnames = ['Name', 'Manager', 'Group', 'Timestamp']

    file_exists = os.path.isfile(csv_filename)

    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            'Name': name,
            'Manager': manager,
            'Group': group,
            'Timestamp': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

def save_face_images(name, manager, group):
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    face_dir = os.path.join("database", name)
    if not os.path.exists(face_dir):
        os.makedirs(face_dir)

    # Save the details in a text file
    with open(os.path.join(face_dir, 'details.txt'), 'w') as f:
        f.write(f"Manager: {manager}\n")
        f.write(f"Group: {group}\n")

    count = 0
    while count < 6:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image from webcam")
            break

        cv2.imshow("Capture Face", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('c'):  # 'c' key to capture an image
            face_path = os.path.join(face_dir, f"{name}_{count + 1}.jpg")
            cv2.imwrite(face_path, frame)
            count += 1
            print(f"Captured image {count}")
            time.sleep(1)  # Pause to capture a distinct image
        elif key == 27:  # ESC key to exit
            break

    cam.release()
    cv2.destroyAllWindows()

    # Update the names and details file
    update_names_and_details(name, manager, group)

def update_names_and_details(name, manager, group):
    details_file = "names_and_details.txt"

    with open(details_file, 'a') as file:
        file.write(f"Name: {name}\n")
        file.write(f"Manager: {manager}\n")
        file.write(f"Group: {group}\n")
        file.write("\n")  # Blank line for separation

def capture_and_recognize():
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    recognized_faces = {}
    start_times = {}

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture image from webcam")
            break

        frame_path = "current_frame.jpg"
        cv2.imwrite(frame_path, frame)

        detected_faces, face_images = faceDetection(frame_path)

        if detected_faces:
            names, confidences = faceRecognition(face_images)
            for name, confidence in zip(names, confidences):
                if name != 'unknown':
                    if name not in recognized_faces:
                        recognized_faces[name] = 0
                        start_times[name] = time.time()
                    else:
                        if time.time() - start_times[name] >= 3:
                            if recognized_faces[name] == 0:
                                recognized_faces[name] += 1
                                # Read the details from the text file
                                face_dir = os.path.join("database", name)
                                with open(os.path.join(face_dir, 'details.txt'), 'r') as f:
                                    details = f.readlines()
                                    manager = details[0].strip().split(": ")[1]
                                    group = details[1].strip().split(": ")[1]
                                save_to_csv(name, manager, group)

            # Draw bounding boxes and labels
            for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_faces):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                if confidences[i] is not None:
                    label = f"{names[i]} ({confidences[i]:.2f})"
                else:
                    label = names[i]
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        else:
            print("No faces detected")

        cv2.imshow("Face Recognition", frame)

        key = cv2.waitKey(30) & 0xFF
        if key == 27:  # ESC key to exit
            break
        elif key == ord('s'):  # 's' key to save the current frame
            # Save the current frame with a timestamp
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            saved_frame_path = f"saved_frame_{timestamp}.jpg"
            cv2.imwrite(saved_frame_path, frame)
            print(f"Frame saved as {saved_frame_path}")

    cam.release()
    cv2.destroyAllWindows()

# Add new face to the database
def add_new_face():
    name = input("Enter the name of the person: ")
    manager = input("Enter manager name: ")
    group = input("Enter group: ")
    save_face_images(name, manager, group)

# Main function
if __name__ == "__main__":
    action = input("Enter 'add' to add a new face or 'recognize' to start real-time recognition: ")
    if action.lower() == 'add':
        add_new_face()
    else:
        capture_and_recognize()



image 1/1 /Users/anirudh/Desktop/DS_environment/F_R/current_frame.jpg: 480x640 1 face, 94.5ms
Speed: 5.1ms preprocess, 94.5ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)


KeyboardInterrupt: 